In [2]:
import numpy as np
import pandas as pd
import shap
import joblib
import os
from sklearn.model_selection import LeaveOneGroupOut
import shap
import pandas as pd

def get_shap_values(conf_name):
    loaded_model = joblib.load('./results/models/RF/'+conf_name+'.pkl')
    df = pd.read_csv("./Data/processed/"+conf_name+".csv", index_col = 0)
    X = df.drop(['target'], axis = 1)
    X = X.rename(columns=lambda s: s.split("/")[-1])
    y = df.iloc[:,-1]

    logo = LeaveOneGroupOut()
    groups = X['instance']
    logo.get_n_splits(X, y, groups)
    dt = []
    for train_index, test_index in logo.split(X, y, groups):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_train_removed = X_train.drop(['instance', 'function'], axis=1)
        for instance in np.unique(np.array(X_train['instance'])):
            loaded_model = loaded_model.fit(X_train_removed, y_train)
            explainer = shap.TreeExplainer(loaded_model)
            data = X_train[X_train['instance']==instance]
            func = np.array(data['function']).reshape(24,1)
            ins = np.array(data['instance']).reshape(24,1)
            data = data.drop(['instance', 'function'], axis=1)
            shap_values = np.concatenate((func, ins, explainer.shap_values(data)), axis=1)
            dt.extend(shap_values)


    df_dt = pd.DataFrame(dt)


    final_shap = []
    for f in [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 20, 21, 22, 23, 24, 2, 3, 4, 5, 6, 7, 8, 9]:
        for i in range(1, 6):
            d = df_dt[(df_dt[0]==f) & (df_dt[1]==i)]
            d = d.drop([d.columns[0], d.columns[1]], axis=1)
            avg = np.average(d, axis = 0)
            final_shap.append(avg)
    X_removed= X.drop(['instance', 'function'], axis=1)
    avg_f = np.average(final_shap, axis=0)
    return avg_f


dims = [5, 30]
configs = [(0,1), (2,3), (4,5), (6,7), (8,9), (10, 11), (12, 13), (14, 15), (16, 17), (18, 19), (270, 271), (540, 541), (542, 543), (810, 811), (812, 813), (814, 815), (816, 817), (1080, 1081)]
budgets = [500, 2000, 5000, 10000, 50000]
ELA_features = ['diff_mean_02','diff_mean_05','diff_mean_10','diff_mean_25','diff_median_02',
'diff_median_05','diff_median_10','diff_median_25','dist_ratio.coeff_var','eps.max','eps.ratio','eps.s',
'expl_var.cor_init','expl_var.cor_x','expl_var.cov_init','expl_var.cov_x','expl_var_PC1.cor_init',
'expl_var_PC1.cor_x','expl_var_PC1.cov_init','expl_var_PC1.cov_x','h.max','kurtosis','lin_simple.adj_r2',
'lin_simple.coef.max','lin_simple.coef.max_by_min','lin_simple.coef.min','lin_simple.intercept','lin_w_interact.adj_r2',
'm0','nb_fitness.cor','nn_nb.cor','nn_nb.mean_ratio','nn_nb.sd_ratio','number_of_peaks','quad_simple.adj_r2',
'quad_simple.cond','quad_w_interact.adj_r2','ratio_mean_02','ratio_mean_05','ratio_mean_10','ratio_mean_25',
'ratio_median_02','ratio_median_05','ratio_median_10','ratio_median_25','skewness']

column_names = ['step_size_adaptation', 'budget']
column_names.extend(ELA_features)
for dim in dims:
    data = []
    for config in configs:
        config_module_on = config[0]
        config_module_off = config[1]
        for budget in budgets:
            conf_on = "dim_"+str(dim)+"_budget_"+str(budget)+"_conf_"+str(config_module_on)
            conf_off = "dim_"+str(dim)+"_budget_"+str(budget)+"_conf_"+str(config_module_off)
            arr_on = ['csa', budget]
            shap_on = get_shap_values(conf_on)
            arr_on.extend(shap_on)
            arr_off = ['psr', budget]
            shap_off = get_shap_values(conf_off)
            arr_off.extend(shap_off)
            data.append(arr_on)
            data.append(arr_off)

    df = pd.DataFrame(data, columns=column_names)
    print(df.head())
    df.to_csv("./Data/classification_data/step_size_adaptation_dim_"+str(dim))


SHAP dim_5_budget_500_conf_0
SHAP dim_5_budget_500_conf_1
SHAP dim_5_budget_2000_conf_0
SHAP dim_5_budget_2000_conf_1
SHAP dim_5_budget_5000_conf_0
SHAP dim_5_budget_5000_conf_1
SHAP dim_5_budget_10000_conf_0
SHAP dim_5_budget_10000_conf_1
SHAP dim_5_budget_50000_conf_0
SHAP dim_5_budget_50000_conf_1
SHAP dim_5_budget_500_conf_2
SHAP dim_5_budget_500_conf_3
SHAP dim_5_budget_2000_conf_2
SHAP dim_5_budget_2000_conf_3
SHAP dim_5_budget_5000_conf_2
SHAP dim_5_budget_5000_conf_3
SHAP dim_5_budget_10000_conf_2
SHAP dim_5_budget_10000_conf_3
SHAP dim_5_budget_50000_conf_2
SHAP dim_5_budget_50000_conf_3
SHAP dim_5_budget_500_conf_4
SHAP dim_5_budget_500_conf_5
SHAP dim_5_budget_2000_conf_4
SHAP dim_5_budget_2000_conf_5
SHAP dim_5_budget_5000_conf_4
SHAP dim_5_budget_5000_conf_5
SHAP dim_5_budget_10000_conf_4
SHAP dim_5_budget_10000_conf_5
SHAP dim_5_budget_50000_conf_4
SHAP dim_5_budget_50000_conf_5
SHAP dim_5_budget_500_conf_6
SHAP dim_5_budget_500_conf_7
SHAP dim_5_budget_2000_conf_6
SHAP d